## Model Building

* **Want:** Saved Model format showld be .h5.
* **Data:** Different slices of [Food101 dataset](https://www.kaggle.com/dansbecker/food-101).
* **Model(s):** [EfficientNetB0](https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB0) backbone's with different output layers.


## Setting up helper functions

In [ ]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Unzip the downloaded file
def unzip_data(filename):
  """
  Utility function to unzip a zipped file.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [ ]:
# Setup data inputs
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, test_dir, image_size=IMG_SIZE):
  """
  Creates a training and test image BatchDataset from train_dir and test_dir.
  """
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  # Note: the test data is the same as the previous experiment, we could
  # skip creating this, but we'll leave this here to practice.
  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [ ]:
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [ ]:
# Setup input shape and base model, freezing the base model layers
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  # Fine-tune?
  base_model.trainable = False

  # Create input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Add in data augmentation Sequential model as a layer
  x = data_augmentation(inputs)

  # Give base_model inputs (after augmentation) and don't train it
  x = base_model(x, training=False)

  # Pool output features of base model
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Put a dense layer on as the output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # Make a model with inputs and outputs
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16711680/16705208 [==============================] - 0s 0us/step


In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=False):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

## Download data

In [ ]:
# Get data
import zipfile

# Download data (10 class subset of Food101 - https://www.kaggle.com/dansbecker/food-101)
# Already formatted in standard image classification directory style
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

--2021-02-14 05:54:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519184959 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M  93.4MB/s    in 5.3s    

2021-02-14 05:54:58 (93.4 MB/s) - ‘10_food_classes_all_data.zip’ saved [519184959/519184959]



In [ ]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 1 images in '10_food_classes_all_data'.
There are 10 directories and 0 images in '10_food_classes_all_data/train'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/ice_cream'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/ramen'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/chicken_wings'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/pizza'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/steak'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/fried_rice'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/hamburger'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/grilled_salmon'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/sushi'.
There are 0 directories and 750 images in '10_food_classes_all_data/train/chicken_curry'.

In [ ]:
# Check the file in 10_food_classes_10_percent
!ls -la 10_food_classes_all_data

total 28
drwxr-xr-x  4 root root 4096 Feb 14 05:54 .
drwxr-xr-x  1 root root 4096 Feb 14 05:54 ..
-rw-r--r--  1 root root 8196 Feb 14 05:54 .DS_Store
drwxr-xr-x 12 root root 4096 Feb 14 05:54 test
drwxr-xr-x 12 root root 4096 Feb 14 05:54 train


In [ ]:
## That one pesky file that always gets stuck
#!rm 10_food_classes_10_percent/.DS_Store

Notice how each of the training directories now has 75 images rather than 750 images. This is key to demonstrating how well transfer learning can perform with less labelled images.

The test directories still have the same amount of images. This means we'll be training on less data but evaluating our models on the same amount of test data.

In [ ]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

Because you're likely to run multiple experiments, it's a good idea to be able to track them in some way.

In our case, our function saves a model's performance logs to a directory named `[dir_name]/[experiment_name]/[current_timestamp]`, where:
* `dir_name` is the overall logs directory
* `experiment_name` is the particular experiment
* `current_timestamp` is the time the experiment started based on Python's [`datetime.datetime().now()`](https://docs.python.org/3/library/datetime.html#datetime.datetime.now)

> 🔑 **Note:** Depending on your use case, the above experimenting tracking naming method may work or you might require something more specific. The good news is, the TensorBoard callback makes it easy to track modelling logs as long as you specify where to track them. So you can get as creative as you like with how you name your experiments, just make sure you or your team can understand them.



## Model 1 (10 classes)

In [ ]:
# Create BatchDataset
train_data, test_data = create_data_loaders(train_dir="10_food_classes_all_data/train/",
                                            test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
# What size is our data?
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [ ]:
# Create model
model_1 = create_model(num_classes=len(train_data.class_names))

# Fit the model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20210210-231226
Epoch 1/5
235/235 [==============================] - 71s 255ms/step - loss: 1.4620 - accuracy: 0.5484 - val_loss: 0.5066 - val_accuracy: 0.8536
Epoch 2/5
235/235 [==============================] - 50s 211ms/step - loss: 0.7201 - accuracy: 0.7819 - val_loss: 0.4265 - val_accuracy: 0.8569
Epoch 3/5
235/235 [==============================] - 47s 199ms/step - loss: 0.6176 - accuracy: 0.8028 - val_loss: 0.4116 - val_accuracy: 0.8618
Epoch 4/5
235/235 [==============================] - 47s 199ms/step - loss: 0.5606 - accuracy: 0.8227 - val_loss: 0.4060 - val_accuracy: 0.8602
Epoch 5/5
235/235 [==============================] - 45s 189ms/step - loss: 0.5435 - accuracy: 0.8308 - val_loss: 0.3677 - val_accuracy: 0.8750


In [ ]:
# Get an image Tensor
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg

--2021-02-10 23:18:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874848 (2.7M) [image/jpeg]
Saving to: ‘03-pizza-dad.jpeg’

03-pizza-dad.jpeg   100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2021-02-10 23:18:11 (27.7 MB/s) - ‘03-pizza-dad.jpeg’ saved [2874848/2874848]



In [ ]:
# Classes our model is trained on
class_names = train_data.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [ ]:
# Preprocess image
pizza_img = load_and_prep_image("03-pizza-dad.jpeg")
pizza_img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 73.625,  76.75 ,  67.125],
        [114.   , 122.   , 101.   ],
        [146.875, 151.875, 129.875],
        ...,
        [ 14.5  ,  17.5  ,  10.5  ],
        [ 14.25 ,  19.25 ,  12.25 ],
        [ 19.75 ,  22.75 ,  15.75 ]],

       [[239.125, 243.625, 246.125],
        [225.375, 232.125, 234.875],
        [240.   , 245.   , 244.5  ],
        ...,
        [ 11.   ,  14.   ,   7.   ],
        [ 20.   ,  23.   ,  16.   ],
        [ 20.875,  25.875,  18.875]],

       [[ 32.5  ,  34.5  ,  31.5  ],
        [ 44.625,  44.5  ,  42.375],
        [ 33.   ,  38.   ,  34.   ],
        ...,
        [  8.75 ,  13.25 ,   6.25 ],
        [ 14.875,  17.875,  10.875],
        [ 13.625,  20.625,  12.625]],

       ...,

       [[ 61.875,  40.875,  19.875],
        [ 60.   ,  42.   ,  22.   ],
        [ 61.   ,  43.   ,  21.   ],
        ...,
        [134.5  ,  96.125,  60.75 ],
        [104.875,  69.375,  43.125],
        [106.25 ,  75.2

In [ ]:
# Make predictions
pizza_expanded = tf.expand_dims(pizza_img, axis=0) # expand image dimensions (224, 224, 3) -> (1, 224, 224, 3)
pred = model_1.predict(pizza_expanded)
pred

array([[8.2119545e-03, 3.3304928e-05, 6.2837714e-04, 1.6367142e-04,
        3.0865319e-06, 5.1294103e-07, 9.9020046e-01, 1.2048778e-04,
        5.5204531e-05, 5.8289926e-04]], dtype=float32)

In [ ]:
# Check the predicted class
class_names[tf.argmax(pred[0])]

'pizza'

## Save Model 1 

We want to save our model in the TensorFlow [`SavedModel`](https://www.tensorflow.org/guide/saved_model) format.

We can do this using: 
`model.save("PATH_TO_SAVED_MODEL")` to create a `SavedModel` object (our trained model).


In [ ]:
# Save model_1
model_1.save("efficientnet_model_1_10_classes.h5")

INFO:tensorflow:Assets written to: efficientnet_model_1_10_classes/assets
